<a href="https://colab.research.google.com/github/lucyzandile/unsupervised-predict-streamlit-template/blob/master/nmf%20model%20and%20preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# data analysis libraries
import pandas as pd
import numpy as np

# Kaggle requirements
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# visualisation libraries
from matplotlib import pyplot as plt
import seaborn as sns
from numpy.random import RandomState

In [5]:

#word cloud
%matplotlib inline
import wordcloud

from wordcloud import WordCloud, STOPWORDS
%matplotlib inline
sns.set()

In [6]:

# visualisation libraries
from matplotlib import pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [7]:
# ML Models
!pip install scikit-surprise
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

     |████████████████████████████████| 11.8 MB 21.9 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1623252 sha256=bc178934bb5c3d4ec583d3fc676ba8c4c76150c3a24bdf60926537c1c1c88739
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [8]:
# ML Pre processing
from surprise.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Hyperparameter tuning
from surprise.model_selection import GridSearchCV

# High performance hyperparameter tuning
#from tune_sklearn import TuneSearchCV
#import warnings
#warnings.filterwarnings("ignore")


In [26]:
#Load data using pandas 

train = pd.read_csv("/train.csv")
test = pd.read_csv("/test.csv")
df_movies = pd.read_csv("/movies 2.csv")
df_tags = pd.read_csv("/tags.csv")
df_links = pd.read_csv("/links.csv")
df_imdb = pd.read_csv("/imdb.csv")
df_genometags = pd.read_csv("/genometags.csv")
df_genomescores = pd.read_csv("/genomescores.csv")
df_samp = pd.read_csv("/sample_submission.csv")

In [18]:
# Loading as Surprise dataframe 
reader = Reader(rating_scale=(0.5, 5))

data = Dataset.load_from_df(train[['userId', 'movieId' ,'rating']], reader)


In [19]:
# Data split 85/15
trainset, testset = train_test_split(data, test_size=0.15)

In [20]:
# Base algorithm
nmf_algo = NMF()


In [21]:
# Fitting our trainset
nmf_algo.fit(trainset)

# Using the 15% testset to make predictions
predictions = nmf_algo.test(testset) 
predictions

test = pd.DataFrame(predictions)

In [22]:
# View the head
test.head()

,uid,iid,r_ui,est,details
0,158553,122916.0,4.5,5,{'was_impossible': False}
1,33520,1272.0,3.0,5,{'was_impossible': False}
2,32317,1969.0,1.0,5,{'was_impossible': False}
3,76844,2329.0,5.0,5,{'was_impossible': False}
4,91338,155288.0,2.5,5,{'was_impossible': False}


In [28]:
#We are trying to predict ratings for every userId / movieId pair, we implement the below list comprehension to achieve this.
ratings_predictions=[nmf_algo.predict(row.userId, row.movieId) for _,row in test.iterrows()]
ratings_predictions

[Prediction(uid=1, iid=2011, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid=4144, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid=5767, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid=6711, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid=7318, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid=8405, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid=8786, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=2, iid=150, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=2, iid=356, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=2, iid=497, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=2, iid=588, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=2, iid=653, r_ui=None, est=5, details={'was_impossible': False}),
 Predicti

In [29]:
# Converting our prediction into a familiar format-Dataframe
df_pred=pd.DataFrame(ratings_predictions)
df_pred

,uid,iid,r_ui,est,details
0,1,2011,None,5.0,{'was_impossible': False}
1,1,4144,None,5.0,{'was_impossible': False}
2,1,5767,None,5.0,{'was_impossible': False}
3,1,6711,None,5.0,{'was_impossible': False}
4,1,7318,None,5.0,{'was_impossible': False}
...,...,...,...,...,...
5000014,162541,4079,None,5.0,{'was_impossible': False}
5000015,162541,4467,None,5.0,{'was_impossible': False}
5000016,162541,4980,None,5.0,{'was_impossible': False}
5000017,162541,5689,None,5.0,{'was_impossible': False}


In [30]:
# Renaming our predictions to original names
df_pred=df_pred.rename(columns={'uid':'userId', 'iid':'movieId','est':'rating'})
df_pred.drop(['r_ui','details'],axis=1,inplace=True)

In [31]:
# Snippet of our ratings
df_pred.head()

,userId,movieId,rating
0,1,2011,5.0
1,1,4144,5.0
2,1,5767,5.0
3,1,6711,5.0
4,1,7318,5.0


In [32]:
# Concatenating userId/movieId into a single Id column.(code has to be run twice to get desired outcome)
df_pred['Id']=df_pred.apply(lambda x:'%s_%s' % (x['userId'],x['movieId']),axis=1)
df_pred['Id']=df_pred.apply(lambda x:'%s_%s' % (x['userId'],x['movieId']),axis=1)

In [33]:

# drop the two features from the dataset userId and movieId
df_pred.drop(['userId', 'movieId'], inplace=True, axis= 1)

In [34]:
df_pred = df_pred[['Id', 'rating']]
df_pred.shape

(5000019, 2)

In [35]:
df_pred.to_csv("nmf_model_base.csv", index=False)

submission of nmf model gives out a score of 1.80